# World Trade: Queries with Spark's SQL API

<b> Spark Web UI: </b> http://localhost:4040/


## Introduction

This notebook contains a number of query examples using Spark's SQL API on the compiled trade dataset, stored in parquet files, showcasing common ETL-like operations and analyses such as generating columns, filtering, converting data types, aggregations, joining tables, pivots, calculating ranks, finding the top 10 of something, etc.


### Data

This dataset consists of monthly data on bilateral trade flows between all EU countries and all other countries on the globe for the last 20 years plus metadata, compiled from data downloaded from [Eurostat's COMEXT database] with scripts in the notebook `convert-data-to-parquet.ipynb`. This dataset is about 60 GB large.


## Imports

In [1]:
# Initialization and configuration
import findspark
findspark.init()

from pyspark import SparkConf, SparkContext, SQLContext
from pyspark.sql import SparkSession

# Spark
from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.sql.window import Window

# Python
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use("seaborn-whitegrid")

## Initialize and configure Spark

In [2]:
# Configuration
conf = SparkConf().setAppName("WorldTrade").setMaster("local[4]")
conf.set("spark.driver.maxResultSize", "2g")
conf.set("spark.driver.memory", "2g")
conf.set("spark.executor.memory", "2g") 
conf.set("spark.executor.pyspark.memory", "2g")

# Initialization
sc = SparkContext(conf=conf)
sqlc = SQLContext(sc)
spark = SparkSession(sc)

## Load and prepare data

The dataset consists of the following DataFrames:
* Fact tables: `flows`
* Dimension tables: `reporters`, `partners`, `products`

### Flows (year 2018, aggregated to yearly values)

Most query examples use this cached DataFrame instead of reading from all parquet files on my SSD drive to speed things up.

In [3]:
flows = spark.read.parquet("data/parquet/*")
flows = flows.withColumn("YEAR", flows["PERIOD"].substr(1, 4).cast(IntegerType()))
flows = flows \
    .select("REPORTER", "REPORTER_ISO", "PARTNER", "PARTNER_ISO", "TRADE_TYPE", "PRODUCT_NC", "FLOW", "YEAR", "VALUE_IN_EUROS") \
    .filter(flows["YEAR"] == 2018) \
    .filter(flows["PRODUCT_NC"] != "TOTAL") \
    .groupBy("YEAR", "REPORTER", "REPORTER_ISO", "PARTNER", "PARTNER_ISO", "TRADE_TYPE", "PRODUCT_NC", "FLOW", "YEAR").agg({"VALUE_IN_EUROS": "sum"}) \
    .groupBy("YEAR", "REPORTER", "REPORTER_ISO", "PARTNER", "PARTNER_ISO", "TRADE_TYPE", "PRODUCT_NC", "YEAR").pivot("FLOW", [1, 2]).sum("sum(VALUE_IN_EUROS)") \
    .withColumnRenamed("1", "IMPORTS").withColumnRenamed("2", "EXPORTS") \
    .cache()

In [4]:
flows.show(10)

+----+--------+------------+-------+-----------+----------+----------+----+-------+-------+
|YEAR|REPORTER|REPORTER_ISO|PARTNER|PARTNER_ISO|TRADE_TYPE|PRODUCT_NC|YEAR|IMPORTS|EXPORTS|
+----+--------+------------+-------+-----------+----------+----------+----+-------+-------+
|2018|       1|          FR|      3|         NL|         I|  94051091|2018|1534458| 573191|
|2018|       1|          FR|      7|         IE|         I|  25223000|2018|   null| 213320|
|2018|       1|          FR|      9|         GR|         I|  10051015|2018| 342378|9670677|
|2018|       1|          FR|     30|         SE|         I|  61142000|2018|  10248| 199622|
|2018|       1|          FR|     54|         LV|         I|  39119019|2018|   2005|  28602|
|2018|       1|          FR|     55|         LT|         I|  87037000|2018|   null|   3781|
|2018|       1|          FR|    212|         TN|         E|  70071190|2018|   6476|   7426|
|2018|       1|          FR|    248|         SN|         E|  16041390|2018|   nu

In [5]:
flows.printSchema()

root
 |-- YEAR: integer (nullable = true)
 |-- REPORTER: integer (nullable = true)
 |-- REPORTER_ISO: string (nullable = true)
 |-- PARTNER: integer (nullable = true)
 |-- PARTNER_ISO: string (nullable = true)
 |-- TRADE_TYPE: string (nullable = true)
 |-- PRODUCT_NC: string (nullable = true)
 |-- YEAR: integer (nullable = true)
 |-- IMPORTS: long (nullable = true)
 |-- EXPORTS: long (nullable = true)



In [6]:
flows.show(10)

+----+--------+------------+-------+-----------+----------+----------+----+-------+-------+
|YEAR|REPORTER|REPORTER_ISO|PARTNER|PARTNER_ISO|TRADE_TYPE|PRODUCT_NC|YEAR|IMPORTS|EXPORTS|
+----+--------+------------+-------+-----------+----------+----------+----+-------+-------+
|2018|       1|          FR|      3|         NL|         I|  94051091|2018|1534458| 573191|
|2018|       1|          FR|      7|         IE|         I|  25223000|2018|   null| 213320|
|2018|       1|          FR|      9|         GR|         I|  10051015|2018| 342378|9670677|
|2018|       1|          FR|     30|         SE|         I|  61142000|2018|  10248| 199622|
|2018|       1|          FR|     54|         LV|         I|  39119019|2018|   2005|  28602|
|2018|       1|          FR|     55|         LT|         I|  87037000|2018|   null|   3781|
|2018|       1|          FR|    212|         TN|         E|  70071190|2018|   6476|   7426|
|2018|       1|          FR|    248|         SN|         E|  16041390|2018|   nu

In [7]:
# Number of reporting countries
flows.select("REPORTER").distinct().count()

28

In [8]:
# Number of trade partners
flows.select("PARTNER").distinct().count()

246

In [9]:
# Number of distinct products
flows.select("PRODUCT_NC").distinct().count()

9965

### Reporters

In [10]:
schema = StructType([
    StructField("REPORTER", IntegerType(), True),
    StructField("START_DATE", StringType(), True),
    StructField("END_DATE", StringType(), True),
    StructField("REPORTER_NAME", StringType(), True),
    StructField("_c1", StringType(), True),
    StructField("_c2", StringType(), True)
])

reporters = spark.read.csv("data/REPORTERS.txt", sep="\t", header=False, schema=schema)
reporters = reporters \
    .select("REPORTER", "REPORTER_NAME") \
    .withColumn("REPORTER_NAME", F.trim(reporters["REPORTER_NAME"])) \
    .cache()   

In [11]:
reporters.count()

29

In [12]:
# Number of reporters
reporters.show(10)

+--------+-------------+
|REPORTER|REPORTER_NAME|
+--------+-------------+
|       1|       France|
|       2|  Belg.-Luxbg|
|       3|  Netherlands|
|       4|   Fr Germany|
|       5|        Italy|
|       6| Utd. Kingdom|
|       7|      Ireland|
|       8|      Denmark|
|       9|       Greece|
|      10|     Portugal|
+--------+-------------+
only showing top 10 rows



### Partners

The operation `partners.filter(partners["END_DATE"] == "31/12/2500")` removes countries or territories that don't exist as legal entities anymore.

In [13]:
schema = StructType([
    StructField("PARTNER", IntegerType(), True),
    StructField("START_DATE", StringType(), True),
    StructField("END_DATE", StringType(), True),
    StructField("PARTNER_NAME", StringType(), True),
    StructField("_c1", StringType(), True),
    StructField("_c2", StringType(), True)
])

partners = spark.read.csv("data/PARTNERS.txt", sep="\t", header=False, schema=schema)
partners = partners.filter(partners["END_DATE"] == "31/12/2500") \
    .withColumn("PARTNER_NAME", F.trim(partners["PARTNER_NAME"])) \
    .select("PARTNER", "PARTNER_NAME") \
    .cache()

In [14]:
partners.show(10)

+-------+--------------+
|PARTNER|  PARTNER_NAME|
+-------+--------------+
|      1|        France|
|      3|   Netherlands|
|      4|       Germany|
|      5|         Italy|
|      6|United Kingdom|
|      7|       Ireland|
|      8|       Denmark|
|      9|        Greece|
|     10|      Portugal|
|     11|         Spain|
+-------+--------------+
only showing top 10 rows



In [15]:
# Number of partners
partners.count()

249

### Product codes (Combined Nomenclature & Harmonized System)

The traded goods in this sample of the dataset are classified according to two systems: The Harmonized System and the Combined Nomenclature. The Harmonized System at the 2-digit level used here distinguishes between about 100 product categories; the Combined Nomenclature at the 8-digit level used here between about 30,000 products. For more on these systems see:
* Harmonized System: https://en.wikipedia.org/wiki/Harmonized_System
* Combined Nomenclature: https://en.wikipedia.org/wiki/Combined_Nomenclature

In [16]:
schema = StructType([
    StructField("PRODUCT_NC", StringType(), True),
    StructField("START_DATE", StringType(), True),
    StructField("END_DATE", StringType(), True),
    StructField("_c1", StringType(), True),
    StructField("PRODUCT_NC_NAME", StringType(), True),
    StructField("_c2", StringType(), True),
    StructField("_c3", StringType(), True)   
])

products = spark.read.csv("data/CN.txt", sep="\t", header=False, schema=schema)
products = products \
    .select("PRODUCT_NC", "PRODUCT_NC_NAME") \
    .withColumn("PRODUCT_NC_NAME", F.trim(products["PRODUCT_NC_NAME"])) \
    .withColumn("HS2", products["PRODUCT_NC"].substr(1, 2)) \
    .cache()

In [17]:
products.show(10)

+----------+--------------------+---+
|PRODUCT_NC|     PRODUCT_NC_NAME|HS2|
+----------+--------------------+---+
|        01|        LIVE ANIMALS| 01|
|      0101|LIVE HORSES, ASSE...| 01|
|    010110|PURE-BRED BREEDIN...| 01|
|  01011010|PURE-BRED BREEDIN...| 01|
|  01011090|PURE-BRED BREEDIN...| 01|
|    010111|PURE-BRED BREEDIN...| 01|
|  01011100|PURE-BRED BREEDIN...| 01|
|    010119|LIVE HORSES (EXCL...| 01|
|  01011910|HORSES FOR SLAUGHTER| 01|
|  01011990|HORSES (EXCL. THO...| 01|
+----------+--------------------+---+
only showing top 10 rows



In [18]:
# Number of product categories (CN8)
products.count()

31154

## Query examples (SQL API)

In [19]:
# Register table views
flows.createOrReplaceTempView("flows")
reporters.createOrReplaceTempView("reporters")
partners.createOrReplaceTempView("partners")
products.createOrReplaceTempView("products")

### Total imports, exports, and trade balance of all EU countries in 2018

In [20]:
# TO BE DONE

### Total German exports, imports, and trade balance in 2018 in bn

In [21]:
query = """
    select
        sum(EXPORTS/1e9) as EXPORTS_BN,
        sum(IMPORTS/1e9) as IMPORTS_BN,
        (sum(EXPORTS) - sum(IMPORTS))/1e9 as BALANCE_BN 
    from flows 
    where 
        REPORTER_ISO = 'DE' and
        YEAR = 2018 
"""
sqlc.sql(query).show()

+-----------+-----------+----------+
| EXPORTS_BN| IMPORTS_BN|BALANCE_BN|
+-----------+-----------+----------+
|1320.840158|1088.541680|232.295922|
+-----------+-----------+----------+



### Top 10 German export destinations in 2018

In [22]:
query = """
    select
        PARTNER_ISO,
        PARTNER_NAME,
        sum(EXPORTS/1e9) as EXPORTS_BN
    from flows 
    join partners on partners.PARTNER = flows.PARTNER
    where 
        REPORTER_ISO = 'DE' and
        YEAR = 2018 
    group by
        PARTNER_ISO,
        PARTNER_NAME
    order by EXPORTS_BN desc
"""
sqlc.sql(query).show(10)

+-----------+--------------------+----------+
|PARTNER_ISO|        PARTNER_NAME|EXPORTS_BN|
+-----------+--------------------+----------+
|         US|United States of ...|114.480834|
|         FR|              France|105.137964|
|         CN|               China| 93.680048|
|         NL|         Netherlands| 84.295506|
|         GB|      United Kingdom| 81.773771|
|         IT|               Italy| 69.716666|
|         AT|             Austria| 63.726222|
|         PL|              Poland| 63.290483|
|         CH|         Switzerland| 53.626724|
|         BE|             Belgium| 44.238112|
+-----------+--------------------+----------+
only showing top 10 rows



### Top 10 countries with the highest trade deficit with Germany in 2018

In [23]:
query = """
    select
        PARTNER_ISO,
        PARTNER_NAME,
        sum(EXPORTS/1e9) as EXPORTS_BN,
        sum(IMPORTS/1e9) as IMPORTS_BN,
        (sum(EXPORTS) - sum(IMPORTS))/1e9 as BALANCE_BN 
    from flows 
    join partners on partners.PARTNER = flows.PARTNER
    where 
        REPORTER_ISO = 'DE' and
        YEAR = 2018 
    group by
        PARTNER_ISO,
        PARTNER_NAME
    order by BALANCE_BN desc
"""
sqlc.sql(query).show(20)

+-----------+--------------------+----------+----------+----------+
|PARTNER_ISO|        PARTNER_NAME|EXPORTS_BN|IMPORTS_BN|BALANCE_BN|
+-----------+--------------------+----------+----------+----------+
|         US|United States of ...|114.480834| 48.559429| 65.921338|
|         GB|      United Kingdom| 81.773771| 40.509202| 41.264567|
|         FR|              France|105.137964| 69.173274| 35.964740|
|         CN|               China| 93.680048| 75.466817| 18.213171|
|         AT|             Austria| 63.726222| 45.955059| 17.771104|
|         ES|               Spain| 44.191083| 32.897243| 11.293842|
|         CH|         Switzerland| 53.626724| 42.857159| 10.769528|
|         SE|              Sweden| 26.219577| 15.966888| 10.252658|
|         IT|               Italy| 69.716666| 60.399467|  9.317248|
|         KR| Korea (Republic of)| 17.774181|  9.191087|  8.583074|
|         AU|           Australia| 10.090758|  1.718844|  8.371927|
|         MX|              Mexico| 13.986662|  6

### EU exports, imports, trade balance

In [24]:
query = """
    select
        sum(EXPORTS/1e9) as EXPORTS_BN,
        sum(IMPORTS/1e9) as IMPORTS_BN,
        (sum(EXPORTS) - sum(IMPORTS))/1e9 as BALANCE_BN 
    from flows 
    where 
        TRADE_TYPE = 'E' and
        YEAR = 2018 
"""
sqlc.sql(query).show()

+-----------+-----------+----------+
| EXPORTS_BN| IMPORTS_BN|BALANCE_BN|
+-----------+-----------+----------+
|1956.318791|1979.205124|-22.875961|
+-----------+-----------+----------+



### Top trade partners per reporter in 2018, ranked by total exports 

In [25]:
# TO BE DONE

### Exports from Argentina to Germany in 2018 by product category (CN8)

In [26]:
query = """
    select
        flows.PRODUCT_NC,
        PRODUCT_NC_NAME,
        sum(IMPORTS/1e6) as IMPORTS_MN
    from flows
    join products on products.PRODUCT_NC = flows.PRODUCT_NC
    where 
        YEAR = 2018 and
        REPORTER_ISO = 'DE' and
        PARTNER_ISO = 'AR'
    group by
        flows.PRODUCT_NC,
        PRODUCT_NC_NAME
    order by IMPORTS_MN desc
"""
sqlc.sql(query).show(10)

+----------+--------------------+----------+
|PRODUCT_NC|     PRODUCT_NC_NAME|IMPORTS_MN|
+----------+--------------------+----------+
|  26030000|COPPER ORES AND C...|198.439966|
|  02013000|FRESH OR CHILLED ...|194.604795|
|  29371900|POLYPEPTIDE HORMO...| 85.198660|
|  51052900|WOOL, COMBED (EXC...| 46.657958|
|  23040000|OILCAKE AND OTHER...| 34.440129|
|  04090000|       NATURAL HONEY| 34.256382|
|  87082990|PARTS AND ACCESSO...| 28.150679|
|  71069100|SILVER, INCL. SIL...| 27.794690|
|  23080040|ACORNS AND HORSE-...| 21.072345|
|  38231990|FATTY ACIDS, INDU...| 19.457899|
+----------+--------------------+----------+
only showing top 10 rows



### Exports from Argentina to Germany in 2018 by product category (HS2)

In [27]:
query = """
    select
        PX1.HS2,
        PX2.PRODUCT_NC_NAME,
        sum(IMPORTS/1e6) as IMPORTS_MN
    from flows as FX
    join products as PX1 on PX1.PRODUCT_NC = FX.PRODUCT_NC
    join products as PX2 on PX1.HS2 = PX2.PRODUCT_NC

    where 
        YEAR = 2018 and
        REPORTER_ISO = 'DE' and
        PARTNER_ISO = 'AR'
    group by
        PX1.HS2,
        PX2.PRODUCT_NC_NAME

    order by IMPORTS_MN desc
"""
sqlc.sql(query).show(10, truncate=True)

+---+--------------------+----------+
|HS2|     PRODUCT_NC_NAME|IMPORTS_MN|
+---+--------------------+----------+
| 26|  ORES, SLAG AND ASH|217.186164|
| 02|MEAT AND EDIBLE M...|198.467026|
| 87|VEHICLES OTHER TH...| 96.472690|
| 29|   ORGANIC CHEMICALS| 87.133348|
| 23|RESIDUES AND WAST...| 55.514820|
| 51|WOOL, FINE OR COA...| 48.437369|
| 22|BEVERAGES, SPIRIT...| 39.100959|
| 04|DAIRY PRODUCE; BI...| 34.256823|
| 84|NUCLEAR REACTORS,...| 34.110421|
| 71|NATURAL OR CULTUR...| 29.720306|
+---+--------------------+----------+
only showing top 10 rows



### Exports from Argentina to the EU in 2018 larger than 80 mn by product category (CN8)

In [28]:
# TO BE DONE

### Monthly imports into to the EU by partner

In [29]:
# TO BE DONE

### Finding trade volumnes based on product descriptions

In [30]:
# TO BE DONE